In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from utils.elasticsearch_utils import ElasticSearchUtils

/home/.local/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
host_info = {
    'host': 'https://corpus.ncsoft.com:9200',
    'http_auth': 'elastic:nlplab',
}

utils = ElasticSearchUtils(**host_info)

In [4]:
index = 'crawler-naver-conversation'

query = {
  '_source': [
    'public_date',
    'source', 
    'sentences.orgnc_sentence',
    'sentences.trsl_orgnc_sentence',
  ]
}

result = []
utils.export(index=index, query=query, result=result)
len(result)

2295

In [5]:
data_list = []

for item in result:
    for doc in item['sentences']:
        doc['date'] = item['public_date']
        doc['source'] = item['source']
        
        data_list.append(doc)
        
df = pd.DataFrame(data_list)

In [6]:
df

,trsl_orgnc_sentence,orgnc_sentence,date,source
0,네가 생각하기에는 네가 어떤 체형을 가졌다고 생각하니?,What kind of body type do you think you have?,20130313,OPIc SQUARE
1,난 내가 좀 통통 한 것 같아. 난 좀 더 말랐으면 좋겠어.,<b>I think I am very chubby</b>. I want to be ...,20130313,OPIc SQUARE
2,너 전혀 통통하지 않아. 넌 표준인걸.,You are not <b>chubby</b> at all. You are just...,20130313,OPIc SQUARE
3,한국에는 마른 여자가 너무 많아.,There are many skinny girls in Korea.,20130313,OPIc SQUARE
4,그래. 한국에서는 뚱뚱한 사람 찾기가 어렵긴 해.,"Yes, I agree with you. There are not many fat ...",20130313,OPIc SQUARE
5,"야 제이슨, 너 정말 피곤해 보여, 무슨 일 있어?","Hey Jason, <b>You look worn-out</b>. What's wr...",20130319,OPIc SQUARE
6,"알아, 공부하느라고 밤을 새웠거든.",I know. I've been <b>staying up all night</b> ...,20130319,OPIc SQUARE
7,내일 기말시험 있니?,"Do you have a <b>final exam</b>, tomorrow?",20130319,OPIc SQUARE
8,"응, 사실 이번 주에 4개 있어.","Yeah, actually I've got four coming up this week.",20130319,OPIc SQUARE
9,너 아직도 제니랑 데이트하니?,<b>Are you still dating Jenny?</b>,20130317,OPIc SQUARE


In [7]:
import re

df['orgnc_sentence'] = df['orgnc_sentence'].apply(lambda x: re.sub('<b>(.+?)</b>', '\g<1>', x))

In [8]:
df.groupby(by=['orgnc_sentence', 'trsl_orgnc_sentence']).size().to_frame()

,,0
orgnc_sentence,trsl_orgnc_sentence,
But we have to take care of it from now on.,그래도 지금부터 관리해야 해.,1
Do you want to go to the movies with me?,나랑 영화 보러 갈래?,1
Don’t be so dramatic. We’ll just order another one.,너무 오버하지 마. 한 잔 더 주문하면 돼.,1
I don’t want to hurt my back again.,허리를 또 다치고 싶지는 않아.,1
It’s a privilege. What do you need me to do today?,영광입니다. 제가 오늘 뭘 하면 될까요?,1
"Let’s lift it together. One, two... Three!","우리 같이 들자. 하나, 둘… 셋",1
Let’s prepare a variety of sample products.,다양한 제품 샘플들을 준비하도록 합시다.,1
My job hours have been cut to part time.,내 일이 파트타임제로 바뀌었어.,1
"Oh, it goes by the lunar calendar.","아, 음력으로 하는 거야.",1


In [9]:
df.to_excel('crawler-naver-conversation.xlsx')

In [ ]:
df.to_json(
    'crawler-naver-conversation.json.bz2',
    force_ascii=False,
    compression='bz2',
    orient='records',
    lines=True,
)

In [ ]:
from uuid import uuid4
from tqdm.autonotebook import tqdm
from utils.elasticsearch_utils import ElasticSearchUtils

host_info = {
    'host': 'https://corpus.ncsoft.com:9200',
    'http_auth': 'elastic:nlplab',
}

index = 'corpus-bitext-conversation'

utils = ElasticSearchUtils(**host_info)

In [ ]:
for i, row in tqdm(bitext_all.iterrows(), total=len(bitext_all)):
    doc = dict(row)
    doc['_id'] = str(uuid4())

    utils.save_document(index=index, document=doc)

utils.flush()    